# TP - HMM
## Import


In [1]:
import numpy as np

## Variables

In [2]:
states = ("pluie","nuage",'soleil')
obs =("marcher","peindre","marcher","travailler")
emit_p = {
    'pluie' :  {'travailler' : 0.6, 'marcher' : 0.1, 'peindre' : 0.3},
    'nuage' :  {'travailler' : 0.3, 'marcher' : 0.5, 'peindre' : 0.2},
    'soleil' : {'travailler' : 0.2, 'marcher' : 0.6, 'peindre' : 0.2}
}
start_p = {
    'pluie' : 0.3,
    'nuage' : 0.5,
    'soleil' : 0.2
}
trans_p = {
    'pluie' : {'pluie' : 0.4, 'nuage' : 0.3, 'soleil' : 0.3},
    'nuage' : {'pluie' : 0.2, 'nuage' : 0.6, 'soleil' : 0.2},
    'soleil' : {'pluie' : 0.1, 'nuage' : 0.1, 'soleil' : 0.8}
}

## Algorithme

In [3]:
def create_mat_res(states, path):
    ret = {}
    for st in states:
        tab = [0 for i in range(len(path))]
        ret.update({st : tab})
    return ret

def hmm(path, states, observations, start_probability, transition_probability, emission_probability):
    mat_res = create_mat_res(states, path)
    mat_path = [[0 for i in range(len(path))] for j in range(len(states))]
    for col_path in range(len(path)):
        obs = path[col_path]

        for etat in states:
            mini = 0
            val = 0
            for etat_rec in states:
                if col_path == 0 :
                    val = start_probability[etat_rec] * emission_probability[etat][obs] * transition_probability[etat_rec][etat]
                else : 
                    val = emission_probability[etat][obs] * transition_probability[etat_rec][etat]

                mini = val if val > mini else mini

            mat_res[etat][col_path] = val

    return mat_res, mat_path

In [7]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    """
    Return the MAP estimate of state trajectory of Hidden Markov Model.

    Parameters
    ----------
    obs : array (T,)
        Observation state sequence. int dtype.
    trans_p : array (K, K)
        State transition matrix. See HiddenMarkovModel.state_transition  for
        details.
    emit_p : array (K, M)
        Emission matrix. See HiddenMarkovModel.emission for details.
    start_p: optional, (K,)
        Initial state probabilities: Pi[i] is the probability x[0] == i. If
        None, uniform initial distribution is assumed (Pi[:] == 1/K).

    Returns
    -------
    x : array (T,)
        Maximum a posteriori probability estimate of hidden state trajectory,
        conditioned on observation sequence y under the model parameters A, B,
        Pi.
    T1: array (K, T)
        the probability of the most likely path so far
    T2: array (K, T)
        the x_j-1 of the most likely path so far
    """
    V = [{}]
    for st in states:
        V[0][st] = {"prob": start_p[st] * emit_p[st][obs[0]], "prev": None}
    # Run Viterbi when t > 0
    for t in range(1, len(obs)):
        V.append({})
        for st in states:
            max_tr_prob = V[t - 1][states[0]]["prob"] * trans_p[states[0]][st]
            prev_st_selected = states[0]
            for prev_st in states[1:]:
                tr_prob = V[t - 1][prev_st]["prob"] * trans_p[prev_st][st]
                if tr_prob > max_tr_prob:
                    max_tr_prob = tr_prob
                    prev_st_selected = prev_st

            max_prob = max_tr_prob * emit_p[st][obs[t]]
            V[t][st] = {"prob": max_prob, "prev": prev_st_selected}

    for line in dptable(V):
        print(line)

    opt = []
    max_prob = 0.0
    previous = None
    # Get most probable state and its backtrack
    for st, data in V[-1].items():
        if data["prob"] > max_prob:
            max_prob = data["prob"]
            best_st = st
    opt.append(best_st)
    previous = best_st

    # Follow the backtrack till the first observation
    for t in range(len(V) - 2, -1, -1):
        opt.insert(0, V[t + 1][previous]["prev"])
        previous = V[t + 1][previous]["prev"]
    
    print ('The steps of states are ' + ' '.join(opt) + ' with highest probability of %s' % max_prob)
    return V


def dptable(V):
    # Print a table of steps from dictionary
    yield " ".join(("%9d" % i) for i in range(len(V)))
    for state in V[0]:
        yield "%.7s: " % state + " ".join("%.7s" % ("%f" % v[state]["prob"]) for v in V)



In [8]:
viterbi(obs, states, start_p, trans_p, emit_p)

        0         1         2         3
pluie : 0.03000 0.01500 0.00060 0.00108
nuage : 0.25000 0.03000 0.00900 0.00162
soleil : 0.12000 0.01920 0.00921 0.00147
The steps of states are nuage nuage nuage nuage with highest probability of 0.0016199999999999997


[{'pluie': {'prob': 0.03, 'prev': None},
  'nuage': {'prob': 0.25, 'prev': None},
  'soleil': {'prob': 0.12, 'prev': None}},
 {'pluie': {'prob': 0.015, 'prev': 'nuage'},
  'nuage': {'prob': 0.03, 'prev': 'nuage'},
  'soleil': {'prob': 0.019200000000000002, 'prev': 'soleil'}},
 {'pluie': {'prob': 0.0006000000000000001, 'prev': 'pluie'},
  'nuage': {'prob': 0.009, 'prev': 'nuage'},
  'soleil': {'prob': 0.009216, 'prev': 'soleil'}},
 {'pluie': {'prob': 0.00108, 'prev': 'nuage'},
  'nuage': {'prob': 0.0016199999999999997, 'prev': 'nuage'},
  'soleil': {'prob': 0.0014745600000000002, 'prev': 'soleil'}}]